# DPF Operators Workshop

This notebook introduces **DPF Operators** - the building blocks for data processing in PyDPF-Core. Operators represent atomic data processing operations that transform input data into output data.

## What are Operators?

Operators are reusable processing units that:
- Take input data (Fields, FieldsContainers, Scoping, etc.)
- Perform a specific operation (calculation, transformation, filtering)
- Produce output data in standard DPF formats

Think of operators as functions that can be chained together to build complex data processing pipelines.

## Setup: Load Model and Initial Data

In [1]:
from ansys.dpf import core as dpf
from ansys.dpf.core import examples
import numpy as np

# Load a transient analysis result file
model = dpf.Model(examples.download_transient_result())

print("Model loaded successfully!")
print(f"Result file: {model.metadata.data_sources.result_key}")
print(f"Number of results: {len(model.metadata.time_freq_support.time_frequencies)}")
print(f"Analysis type: {model.metadata.result_info.analysis_type}")

Model loaded successfully!
Result file: rst
Number of results: 35
Analysis type: static


## Basic Operator Usage

### Understanding Operator Structure

Every operator has:
- **Inputs**: Data or parameters you provide (accessed via `operator.inputs`)
- **Outputs**: Results produced by the operator (accessed via `operator.outputs`)
- **Connection**: Methods to link operators together

| **Component** | **Description** | **Example** |
|---------------|-----------------|-------------|
| `inputs` | Input pins to connect data | `op.inputs.time_scoping([1])` |
| `outputs` | Output pins to get results | `op.outputs.fields_container()` |
| `connect()` | Connect data to input pins | `op.connect(0, field)` |

### Example 1: Using the Displacement Operator

In [2]:
# Create a displacement operator
displacement_op = dpf.operators.result.displacement()

# Connect inputs
displacement_op.inputs.data_sources(model.metadata.data_sources)
displacement_op.inputs.time_scoping([1])  # Get displacement at first time step

# Execute and get output
displacement_fc = displacement_op.outputs.fields_container()

# Display results
print("Displacement Operator Results:")
print(f"  Number of fields: {len(displacement_fc)}")
print(f"  Field location: {displacement_fc[0].location}")
print(f"  Field unit: {displacement_fc[0].unit}")
print(f"  Number of nodes: {len(displacement_fc[0].scoping)}")
print(f"  Max displacement: {np.max(np.linalg.norm(displacement_fc[0].data, axis=1)):.6f} {displacement_fc[0].unit}")

Displacement Operator Results:
  Number of fields: 1
  Field location: Nodal
  Field unit: m
  Number of nodes: 3820
  Max displacement: 0.000000 m

  Number of fields: 1
  Field location: Nodal
  Field unit: m
  Number of nodes: 3820
  Max displacement: 0.000000 m


### Example 2: Using the Stress Operator

In [3]:
# Create a stress operator
stress_op = dpf.operators.result.stress()

# Connect inputs
stress_op.inputs.data_sources(model.metadata.data_sources)
stress_op.inputs.time_scoping([1, 2, 3])  # Get stress at first 3 time steps

# Execute and get output
stress_fc = stress_op.outputs.fields_container()

# Display results
print("Stress Operator Results:")
print(f"  Number of fields (time steps): {len(stress_fc)}")
print(f"  Field location: {stress_fc[0].location}")
print(f"  Field unit: {stress_fc[0].unit}")
print(f"  Data shape per element: {stress_fc[0].data.shape}")
print()

# Show stress for each time step
for i, field in enumerate(stress_fc):
    max_stress = np.max(np.abs(field.data))
    print(f"  Time step {i+1}: Max stress component = {max_stress:.2f} {field.unit}")

Stress Operator Results:
  Number of fields (time steps): 3
  Field location: ElementalNodal
  Field unit: Pa
  Data shape per element: (5720, 6)

  Time step 1: Max stress component = 0.00 Pa
  Time step 2: Max stress component = 47599.05 Pa
  Time step 3: Max stress component = 73779.55 Pa


## Chaining Operators

One of the most powerful features of DPF is the ability to chain operators together. The output of one operator becomes the input to another.

### Example 3: Computing Von Mises Stress

In [4]:
# Step 1: Get stress tensor
stress_op = dpf.operators.result.stress()
stress_op.inputs.data_sources(model.metadata.data_sources)
stress_op.inputs.time_scoping([1])

# Step 2: Compute Von Mises equivalent stress from stress tensor
von_mises_op = dpf.operators.invariant.von_mises_eqv_fc()

# Chain operators: connect stress output to von_mises input
von_mises_op.inputs.fields_container(stress_op.outputs.fields_container)

# Get the result
von_mises_fc = von_mises_op.outputs.fields_container()
von_mises_field = von_mises_fc[0]

print("Von Mises Stress Calculation:")
print(f"  Input: Stress tensor ({stress_op.outputs.fields_container()[0].data.shape})")
print(f"  Output: Von Mises scalar ({von_mises_field.data.shape})")
print(f"  Unit: {von_mises_field.unit}")
print(f"  Location: {von_mises_field.location}")
print()
print(f"  Min Von Mises stress: {np.min(von_mises_field.data):.2f} {von_mises_field.unit}")
print(f"  Max Von Mises stress: {np.max(von_mises_field.data):.2f} {von_mises_field.unit}")
print(f"  Avg Von Mises stress: {np.mean(von_mises_field.data):.2f} {von_mises_field.unit}")

Von Mises Stress Calculation:
  Input: Stress tensor ((5720, 6))
  Output: Von Mises scalar ((5720,))
  Unit: Pa
  Location: ElementalNodal

  Min Von Mises stress: 0.00 Pa
  Max Von Mises stress: 0.00 Pa
  Avg Von Mises stress: 0.00 Pa


### Example 4: Location Conversion (Elemental to Nodal)

In [5]:
# Von Mises stress is at elemental nodal location
# Let's convert it to pure nodal location for visualization

# Create averaging operator to convert to nodal
to_nodal_op = dpf.operators.averaging.to_nodal_fc()

# Connect von mises output to averaging input
to_nodal_op.inputs.fields_container(von_mises_op.outputs.fields_container)

# Get nodal result
nodal_von_mises_fc = to_nodal_op.outputs.fields_container()
nodal_von_mises = nodal_von_mises_fc[0]

print("Location Conversion:")
print(f"  Original location: {von_mises_field.location}")
print(f"  Original entities: {len(von_mises_field.scoping)}")
print()
print(f"  Converted location: {nodal_von_mises.location}")
print(f"  Converted entities: {len(nodal_von_mises.scoping)}")
print()
print(f"  Max nodal Von Mises: {np.max(nodal_von_mises.data):.2f} {nodal_von_mises.unit}")

Location Conversion:
  Original location: ElementalNodal
  Original entities: 715

  Converted location: Nodal
  Converted entities: 3812

  Max nodal Von Mises: 0.00 Pa


## Mathematical Operators

DPF provides operators for mathematical operations on fields.

### Example 5: Scaling a Field

In [8]:
# Get displacement field
disp_op = dpf.operators.result.displacement()
disp_op.inputs.data_sources(model.metadata.data_sources)
disp_op.inputs.time_scoping([30])
disp_fc = disp_op.outputs.fields_container()

# Scale the displacement by a factor (e.g., for visualization)
scale_op = dpf.operators.math.scale_fc()
scale_op.inputs.fields_container(disp_fc)
scale_op.inputs.ponderation(10.0)  # Scale factor

# Get scaled result
scaled_disp_fc = scale_op.outputs.fields_container()

print("Scaling Operation:")
print(f"  Original max displacement: {np.max(np.linalg.norm(disp_fc[0].data, axis=1)):.6f}")
print(f"  Scale factor: 10.0")
print(f"  Scaled max displacement: {np.max(np.linalg.norm(scaled_disp_fc[0].data, axis=1)):.6f}")
print()
print("  First node original:", disp_fc[0].data[0])
print("  First node scaled:", scaled_disp_fc[0].data[0])

Scaling Operation:
  Original max displacement: 0.041883
  Scale factor: 10.0
  Scaled max displacement: 0.418829

  First node original: [ 9.42062447e-03 -1.54279501e-02 -3.79754021e-05]
  First node scaled: [ 0.09420624 -0.1542795  -0.00037975]


### Example 6: Computing Field Magnitude (Norm)

In [9]:
# Displacement is a vector field [x, y, z]
# Let's compute its magnitude (norm)

norm_op = dpf.operators.math.norm_fc()
norm_op.inputs.fields_container(disp_fc)

# Get magnitude field
disp_magnitude_fc = norm_op.outputs.fields_container()
disp_magnitude = disp_magnitude_fc[0]

print("Norm Calculation:")
print(f"  Input field shape: {disp_fc[0].data.shape} (vector)")
print(f"  Output field shape: {disp_magnitude.data.shape} (scalar)")
print()
print("Example for first node:")
print(f"  Displacement vector: {disp_fc[0].data[0]}")
print(f"  Magnitude: {disp_magnitude.data[0]:.6f} {disp_magnitude.unit}")
print(f"  Verification: sqrt({disp_fc[0].data[0][0]:.6f}² + {disp_fc[0].data[0][1]:.6f}² + {disp_fc[0].data[0][2]:.6f}²) = {np.linalg.norm(disp_fc[0].data[0]):.6f}")
print()
print(f"  Max displacement magnitude: {np.max(disp_magnitude.data):.6f} {disp_magnitude.unit}")

Norm Calculation:
  Input field shape: (3820, 3) (vector)
  Output field shape: (3820,) (scalar)

Example for first node:
  Displacement vector: [ 9.42062447e-03 -1.54279501e-02 -3.79754021e-05]
  Magnitude: 0.018077 m
  Verification: sqrt(0.009421² + -0.015428² + -0.000038²) = 0.018077

  Max displacement magnitude: 0.041883 m


### Example 7: Adding Two Fields

In [11]:
# Get displacement at two different time steps
disp_op_t1 = dpf.operators.result.displacement()
disp_op_t1.inputs.data_sources(model.metadata.data_sources)
disp_op_t1.inputs.time_scoping([30])

disp_op_t2 = dpf.operators.result.displacement()
disp_op_t2.inputs.data_sources(model.metadata.data_sources)
disp_op_t2.inputs.time_scoping([34])

# Add the two displacement fields
add_op = dpf.operators.math.add()
add_op.inputs.fieldA(disp_op_t1.outputs.fields_container()[0])
add_op.inputs.fieldB(disp_op_t2.outputs.fields_container()[0])

# Get result
sum_field = add_op.outputs.field()

print("Field Addition:")
print(f"  Field A (t=30) max: {np.max(np.linalg.norm(disp_op_t1.outputs.fields_container()[0].data, axis=1)):.6f}")
print(f"  Field B (t=34) max: {np.max(np.linalg.norm(disp_op_t2.outputs.fields_container()[0].data, axis=1)):.6f}")
print(f"  Sum field max: {np.max(np.linalg.norm(sum_field.data, axis=1)):.6f}")
print()
print("First node verification:")
print(f"  A: {disp_op_t1.outputs.fields_container()[0].data[0]}")
print(f"  B: {disp_op_t2.outputs.fields_container()[0].data[0]}")
print(f"  A+B: {sum_field.data[0]}")

Field Addition:
  Field A (t=30) max: 0.041883
  Field B (t=34) max: 0.060771
  Sum field max: 0.095839

First node verification:
  A: [ 9.42062447e-03 -1.54279501e-02 -3.79754021e-05]
  B: [ 2.39725373e-02 -1.16947735e-02 -8.09525206e-05]
  A+B: [ 0.03339316 -0.02712272 -0.00011893]


## Filtering Operators

Filtering operators allow you to extract subsets of data based on various criteria.

### Example 8: Component Selector (Extracting X, Y, or Z Components)

In [12]:
# Extract only the Z component of displacement
component_selector_op = dpf.operators.logic.component_selector_fc()
component_selector_op.inputs.fields_container(disp_fc)
component_selector_op.inputs.component_number(2)  # 0=X, 1=Y, 2=Z

# Get Z component
z_displacement_fc = component_selector_op.outputs.fields_container()
z_displacement = z_displacement_fc[0]

print("Component Selection:")
print(f"  Original field shape: {disp_fc[0].data.shape} (3D vector: X, Y, Z)")
print(f"  Selected component: Z (index 2)")
print(f"  Result field shape: {z_displacement.data.shape} (scalar)")
print()
print("First 5 nodes:")
print("  Full displacement [X, Y, Z]:")
for i in range(5):
    print(f"    Node {i}: {disp_fc[0].data[i]}")
print()
print("  Z component only:")
for i in range(5):
    print(f"    Node {i}: {z_displacement.data[i]:.6f}")

Component Selection:
  Original field shape: (3820, 3) (3D vector: X, Y, Z)
  Selected component: Z (index 2)
  Result field shape: (3820,) (scalar)

First 5 nodes:
  Full displacement [X, Y, Z]:
    Node 0: [ 9.42062447e-03 -1.54279501e-02 -3.79754021e-05]
    Node 1: [ 2.51963302e-02 -2.96128546e-02 -5.99682523e-05]
    Node 2: [-0.00312272 -0.03875578 -0.00014406]
    Node 3: [-3.34324926e-03 -2.41367858e-02 -9.22574065e-05]
    Node 4: [ 1.73084773e-02 -2.25204024e-02 -4.89718272e-05]

  Z component only:
    Node 0: -0.000038
    Node 1: -0.000060
    Node 2: -0.000144
    Node 3: -0.000092
    Node 4: -0.000049


### Example 9: Scoping - Extracting Data for Specific Nodes

In [13]:
# Create a scoping for specific nodes we want to extract
my_scoping = dpf.Scoping()
my_scoping.ids = [1, 10, 20, 30, 40]  # Specific node IDs
my_scoping.location = dpf.locations.nodal

# Use the displacement operator with mesh scoping
disp_scoped_op = dpf.operators.result.displacement()
disp_scoped_op.inputs.data_sources(model.metadata.data_sources)
disp_scoped_op.inputs.time_scoping([30])
disp_scoped_op.inputs.mesh_scoping(my_scoping)

# Get result (only for specified nodes)
disp_scoped_fc = disp_scoped_op.outputs.fields_container()
disp_scoped = disp_scoped_fc[0]

print("Scoping Filter:")
print(f"  Original field nodes: {len(disp_fc[0].scoping)}")
print(f"  Requested node IDs: {my_scoping.ids}")
print(f"  Result field nodes: {len(disp_scoped.scoping)}")
print(f"  Result node IDs: {disp_scoped.scoping.ids}")
print()
print("Displacement for selected nodes:")
for i, node_id in enumerate(disp_scoped.scoping.ids):
    print(f"  Node {node_id}: {disp_scoped.data[i]}")

Scoping Filter:
  Original field nodes: 3820
  Requested node IDs: [ 1 10 20 30 40]
  Result field nodes: 4
  Result node IDs: [10 20 30 40]

Displacement for selected nodes:
  Node 10: [-1.62417724e-02 -2.81583557e-03  8.07592503e-05]
  Node 20: [-0.00364277 -0.02425495  0.00015316]
  Node 30: [ 1.60383816e-02  1.49132623e-02 -7.05449586e-05]
  Node 40: [-9.23274624e-03  1.08242824e-02  1.83059322e-05]


## Min/Max Operators

Extract minimum and maximum values from fields.

### Example 10: Finding Maximum Displacement

In [14]:
# First get displacement magnitude
norm_op = dpf.operators.math.norm_fc()
norm_op.inputs.fields_container(disp_fc)
disp_mag_fc = norm_op.outputs.fields_container()

# Find minimum and maximum
min_max_op = dpf.operators.min_max.min_max_fc()
min_max_op.inputs.fields_container(disp_mag_fc)

# Get results
min_field = min_max_op.outputs.field_min()
max_field = min_max_op.outputs.field_max()

print("Min/Max Displacement Magnitude:")
print(f"  Minimum value: {min_field.data[0]:.6f} {min_field.unit}")
print(f"  Minimum at node: {min_field.scoping.ids[0]}")
print()
print(f"  Maximum value: {max_field.data[0]:.6f} {max_field.unit}")
print(f"  Maximum at node: {max_field.scoping.ids[0]}")
print()
print("Verification with numpy:")
print(f"  Numpy min: {np.min(disp_mag_fc[0].data):.6f}")
print(f"  Numpy max: {np.max(disp_mag_fc[0].data):.6f}")

Min/Max Displacement Magnitude:
  Minimum value: 0.000000 m
  Minimum at node: 0

  Maximum value: 0.041883 m
  Maximum at node: 0

Verification with numpy:
  Numpy min: 0.000000
  Numpy max: 0.041883


## Building Complex Processing Chains

Let's combine multiple operators to perform a complex analysis.

### Example 11: Complete Von Mises Analysis Pipeline

In [18]:
"""
Pipeline: Stress → Von Mises → Nodal Average → Min/Max
This is a typical workflow for structural analysis
"""

# Step 1: Get stress tensor for multiple time steps
stress_op = dpf.operators.result.stress()
stress_op.inputs.data_sources(model.metadata.data_sources)
stress_op.inputs.time_scoping([1, 2, 3])

# Step 2: Compute Von Mises equivalent stress
von_mises_op = dpf.operators.invariant.von_mises_eqv_fc()
von_mises_op.inputs.fields_container(stress_op.outputs.fields_container)

# Step 3: Convert to nodal location for better visualization
to_nodal_op = dpf.operators.averaging.to_nodal_fc()
to_nodal_op.inputs.fields_container(von_mises_op.outputs.fields_container)

# Step 4: Find min/max across all time steps
min_max_op = dpf.operators.min_max.min_max_fc()
min_max_op.inputs.fields_container(to_nodal_op.outputs.fields_container)

# Get final results
min_stress = min_max_op.outputs.field_min()
max_stress = min_max_op.outputs.field_max()

print("Complete Von Mises Analysis Pipeline:")
print("=" * 60)
print("Pipeline: Stress → Von Mises → Nodal → Min/Max")
print()
print(f"Number of fields processed: {len(to_nodal_op.outputs.fields_container())}")
print()
print("Results across all time steps:")
print(f"  Minimum Von Mises stress: {min_stress.data[0]:.2f} {min_stress.unit} at node {min_stress.scoping.ids[0]}")
print(f"  Maximum Von Mises stress: {max_stress.data[0]:.2f} {max_stress.unit} at node {max_stress.scoping.ids[0]}")
print()
print("Per time step:")
nodal_vm_fc = to_nodal_op.outputs.fields_container()
for i, field in enumerate(nodal_vm_fc):
    print(f"  Time step {i+1}: Max = {np.max(field.data):.2f} {field.unit}")

Complete Von Mises Analysis Pipeline:
Pipeline: Stress → Von Mises → Nodal → Min/Max

Number of fields processed: 3

Results across all time steps:
  Minimum Von Mises stress: 0.00 Pa at node 0
  Maximum Von Mises stress: 0.00 Pa at node 0

Per time step:
  Time step 1: Max = 0.00 Pa
  Time step 2: Max = 62351.09 Pa
  Time step 3: Max = 96965.34 Pa


## Operator Categories

DPF provides operators organized into functional categories:

| **Category** | **Purpose** | **Example Operators** |
|--------------|-------------|----------------------|
| `result` | Extract results from files | `displacement`, `stress`, `temperature`, `elastic_strain` |
| `math` | Mathematical operations | `add`, `scale`, `norm_fc`, `dot` |
| `averaging` | Spatial averaging | `to_nodal_fc`, `to_elemental_fc` |
| `logic` | Data filtering/selection | `component_selector_fc`, `merge_fields` |
| `invariant` | Tensor invariants | `von_mises_eqv_fc`, `principal_invariants` |
| `min_max` | Min/Max operations | `min_max_fc`, `min_max_over_time` |
| `utility` | Utility operations | `forward_field`, `merge_fields_containers` |

### Exploring Available Operators

In [19]:
# You can explore available operators
print("Sample of available result operators:")
result_ops = ['displacement', 'stress', 'elastic_strain', 'temperature', 
              'velocity', 'acceleration', 'reaction_force']
for op_name in result_ops:
    print(f"  - dpf.operators.result.{op_name}()")

print("\nSample of available math operators:")
math_ops = ['add', 'subtract', 'scale', 'norm_fc', 'dot', 'generalized_inner_product']
for op_name in math_ops:
    print(f"  - dpf.operators.math.{op_name}()")

print("\nSample of available averaging operators:")
avg_ops = ['to_nodal_fc', 'to_elemental_fc', 'elemental_mean']
for op_name in avg_ops:
    print(f"  - dpf.operators.averaging.{op_name}()")

print("\nSample of available invariant operators:")
inv_ops = ['von_mises_eqv_fc', 'principal_invariants', 'hydrostatic_pressure']
for op_name in inv_ops:
    print(f"  - dpf.operators.invariant.{op_name}()")

Sample of available result operators:
  - dpf.operators.result.displacement()
  - dpf.operators.result.stress()
  - dpf.operators.result.elastic_strain()
  - dpf.operators.result.temperature()
  - dpf.operators.result.velocity()
  - dpf.operators.result.acceleration()
  - dpf.operators.result.reaction_force()

Sample of available math operators:
  - dpf.operators.math.add()
  - dpf.operators.math.subtract()
  - dpf.operators.math.scale()
  - dpf.operators.math.norm_fc()
  - dpf.operators.math.dot()
  - dpf.operators.math.generalized_inner_product()

Sample of available averaging operators:
  - dpf.operators.averaging.to_nodal_fc()
  - dpf.operators.averaging.to_elemental_fc()
  - dpf.operators.averaging.elemental_mean()

Sample of available invariant operators:
  - dpf.operators.invariant.von_mises_eqv_fc()
  - dpf.operators.invariant.principal_invariants()
  - dpf.operators.invariant.hydrostatic_pressure()


## Key Takeaways

### Operator Workflow
1. **Create** the operator: `op = dpf.operators.category.operator_name()`
2. **Connect inputs**: `op.inputs.input_name(value)`
3. **Execute** (automatic when accessing outputs)
4. **Get outputs**: `result = op.outputs.output_name()`

### Connection Patterns
- **Direct data**: `op.inputs.field(my_field)`
- **Operator chaining**: `op2.inputs.field(op1.outputs.field())`
- **Multiple time steps**: `op.inputs.time_scoping([1, 2, 3])`
- **Specific entities**: `op.inputs.mesh_scoping(my_scoping)`

### Common Operations
- **Result extraction**: Get displacement, stress, strain, etc.
- **Math operations**: Add, scale, compute norms
- **Invariants**: Von Mises, principal stresses
- **Filtering**: Component selection, scoping
- **Location conversion**: Nodal ↔ Elemental
- **Min/Max**: Find extreme values

### Best Practices
- Chain operators to build processing pipelines
- Use appropriate scoping to reduce data processing
- Convert to nodal location for visualization
- Leverage categories to organize your workflows

## Practice Exercise

Try creating your own operator chain! Here's a suggested exercise:

**Task**: Create a pipeline that:
1. Extracts displacement for time steps 2, 3, and 4
2. Computes the magnitude (norm) of the displacement
3. Converts to nodal location
4. Finds the maximum displacement across all time steps

Use the cell below to implement this!

In [ ]:
# Your code here - implement the exercise!

# Step 1: Extract displacement

# Step 2: Compute magnitude

# Step 3: Convert to nodal

# Step 4: Find maximum

# Print results

### Solution (Try the exercise first before looking!)

In [21]:
# Solution to the exercise

# Step 1: Extract displacement for time steps 2, 3, 4
disp_exercise_op = dpf.operators.result.displacement()
disp_exercise_op.inputs.data_sources(model.metadata.data_sources)
disp_exercise_op.inputs.time_scoping([2, 3, 4])

# Step 2: Compute magnitude (norm)
norm_exercise_op = dpf.operators.math.norm_fc()
norm_exercise_op.inputs.fields_container(disp_exercise_op.outputs.fields_container)

# Step 3: Convert to nodal location
nodal_exercise_op = dpf.operators.averaging.to_nodal_fc()
nodal_exercise_op.inputs.fields_container(norm_exercise_op.outputs.fields_container)

# Step 4: Find maximum across all time steps
max_exercise_op = dpf.operators.min_max.min_max_fc()
max_exercise_op.inputs.fields_container(nodal_exercise_op.outputs.fields_container)

# Get and print results
max_disp = max_exercise_op.outputs.field_max()

print("Exercise Solution Results:")
print("=" * 60)
print(f"Maximum displacement: {max_disp.data[0]:.6f} {max_disp.unit}")
print(f"Located at node: {max_disp.scoping.ids[0]}")
print()
print("Displacement magnitude per time step:")
nodal_fc = nodal_exercise_op.outputs.fields_container()
for i, field in enumerate(nodal_fc):
    print(f"  Time step {i+2}: Max = {np.max(field.data):.6f} {field.unit}")

Exercise Solution Results:
Maximum displacement: 0.000627 m
Located at node: 0

Displacement magnitude per time step:
  Time step 2: Max = 0.000627 m
  Time step 3: Max = 0.002509 m
  Time step 4: Max = 0.005642 m
